In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from object_detection.utils import config_util
from google.protobuf import text_format
from object_detection.protos import pipeline_pb2
# from google.protobuf import text_format
# from keras.models import Sequential
# from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, Lambda, MaxPool2D, BatchNormalization
# from keras.utils import to_categorical
# from keras.preprocessing.image import ImageDataGenerator
# from keras.optimizers import RMSprop
# from sklearn.preprocessing import LabelEncoder
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import confusion_matrix
# from sklearn.metrics import accuracy_score
# import xml.etree.ElementTree as ET
# import sklearn
# import itertools
# import cv2
# import scipy
import os
import csv
# import matplotlib.pyplot as plt
# %matplotlib inline

# Устанавливаем папки

In [2]:
fish_name = input("Напишите с какой рыбой работаем:")
if fish_name.lower() == 'карп':
    LABEL_MAP_NAME = 'label_map_karp.pbtxt'
    name = 'karp'
elif fish_name.lower() == 'осетр':
    LABEL_MAP_NAME = 'label_map_osetr.pbtxt'
    name = 'osetr'
else:
    print('Пока нет фотографий для такой рыбы')

Напишите с какой рыбой работаем: карп


In [3]:
CUSTOM_MODEL_NAME = 'my_ssd_mobnet' 
PRETRAINED_MODEL_NAME = 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8'
PRETRAINED_MODEL_URL = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz'
TF_RECORD_SCRIPT_NAME = 'generate_tfrecord.py'

In [4]:
paths = {
    'WORKSPACE_PATH': os.path.join('Tensorflow', 'workspace'),
    'SCRIPTS_PATH': os.path.join('Tensorflow','scripts'),
    'APIMODEL_PATH': os.path.join('Tensorflow','models'),
    'ANNOTATION_PATH': os.path.join('Tensorflow', 'workspace','annotations'),
    'IMAGE_PATH': os.path.join('Tensorflow', 'workspace','data'),
    'MODEL_PATH': os.path.join('Tensorflow', 'workspace','models'),
    'PRETRAINED_MODEL_PATH': os.path.join('Tensorflow', 'workspace','pre-trained-models'),
    'CHECKPOINT_PATH': os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, name + '_checkpoints'), 
    'OUTPUT_PATH': os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'export'), 
    'TFJS_PATH':os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'tfjsexport'), 
    'TFLITE_PATH':os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'tfliteexport'), 
    'PROTOC_PATH':os.path.join('Tensorflow','protoc')
 }

In [5]:
files = {
    'PIPELINE_CONFIG':os.path.join(paths['CHECKPOINT_PATH'], name + '_pipeline.config'),
    'TF_RECORD_SCRIPT': os.path.join(paths['SCRIPTS_PATH'], TF_RECORD_SCRIPT_NAME), 
    'LABELMAP': os.path.join(paths['ANNOTATION_PATH'], LABEL_MAP_NAME)
}

In [6]:
for path in paths.values():
    if not os.path.exists(path):
        if os.name == 'posix':
            !mkdir -p {path}
        if os.name == 'nt':
            !mkdir {path}

In [7]:
if not os.path.exists(os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection')):
    !git clone https://github.com/tensorflow/models {paths['APIMODEL_PATH']}

Лучше запускать через терминал (нужно активировать venv)

In [8]:
# # Install Tensorflow Object Detection 
# if os.name=='posix':  
#     # !sudo apt-get install protobuf-compiler
#     # !cd Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && cp object_detection/packages/tf2/setup.py . && python3 -m pip install . 
#     ...
# if os.name=='nt':
#     url="https://github.com/protocolbuffers/protobuf/releases/download/v3.15.6/protoc-3.15.6-win64.zip"
#     wget.download(url)
#     !move protoc-3.15.6-win64.zip {paths['PROTOC_PATH']}
#     !cd {paths['PROTOC_PATH']} && tar -xf protoc-3.15.6-win64.zip
#     os.environ['PATH'] += os.pathsep + os.path.abspath(os.path.join(paths['PROTOC_PATH'], 'bin'))   
#     !cd Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && copy object_detection\\packages\\tf2\\setup.py setup.py && python setup.py build && python setup.py install
#     !cd Tensorflow/models/research/slim && pip install -e . 

## Проверяем все ли библиотеки скачались

In [1]:
VERIFICATION_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'builders', 'model_builder_tf2_test.py')
# Verify Installation
%run {VERIFICATION_SCRIPT}

NameError: name 'os' is not defined

# Подгружаем модель

In [10]:
if os.name =='posix':
    !wget {PRETRAINED_MODEL_URL}
    !mv {PRETRAINED_MODEL_NAME+'.tar.gz'} {paths['PRETRAINED_MODEL_PATH']}
    !cd {paths['PRETRAINED_MODEL_PATH']} && tar -zxvf {PRETRAINED_MODEL_NAME+'.tar.gz'}
if os.name == 'nt':
    wget.download(PRETRAINED_MODEL_URL)
    !move {PRETRAINED_MODEL_NAME+'.tar.gz'} {paths['PRETRAINED_MODEL_PATH']}
    !cd {paths['PRETRAINED_MODEL_PATH']} && tar -zxvf {PRETRAINED_MODEL_NAME+'.tar.gz'}

--2024-06-13 17:09:27--  http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 216.58.207.251, 216.58.211.27, 142.250.74.59, ...
Connecting to download.tensorflow.org (download.tensorflow.org)|216.58.207.251|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20515344 (20M) [application/x-tar]
Saving to: ‘ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz’

ssd_mobilenet_v2_fp 100%[===================>]  19.56M  6.82MB/s    in 2.9s    

2024-06-13 17:09:31 (6.82 MB/s) - ‘ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz’ saved [20515344/20515344]

ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0.data-00000-of-00001
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/checkpoint
ssd_mobilenet_v2_fpnlite_320x320_

Копируем конфиг в тренировочную папку

In [11]:
if os.name =='posix':
    !cp {os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'pipeline.config')} {os.path.join(paths['CHECKPOINT_PATH'], name + '_pipeline.config')}
if os.name == 'nt':
    !copy {os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'pipeline.config')} {os.path.join(paths['CHECKPOINT_PATH'])}

# Обновляем конфиг под наши данные

In [16]:
def update_pipeline_config(config_path, label_map_path, train_record_path, test_record_path, checkpoint_path):
    pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()

    # Read the existing pipeline config
    with open(config_path, 'r') as f:
        proto_str = f.read()
        proto_str = proto_str.replace("fine_tune_checkpoint_version: V2", "")
        text_format.Merge(proto_str, pipeline_config)


    # Update the paths
    pipeline_config.model.ssd.num_classes = label_count
    pipeline_config.train_config.batch_size = 4
    pipeline_config.train_config.fine_tune_checkpoint = os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'checkpoint', 'ckpt-0')
    pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
    pipeline_config.train_input_reader.label_map_path= files['LABELMAP']
    pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [os.path.join(paths['ANNOTATION_PATH'], name + '_train.record')]
    pipeline_config.eval_input_reader[0].label_map_path = files['LABELMAP']
    pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [os.path.join(paths['ANNOTATION_PATH'], name + '_test.record')]

    # Write the updated pipeline config back to the file
    with open(config_path, 'w') as f:
        f.write(text_format.MessageToString(pipeline_config))

In [18]:
with open(files['LABELMAP']) as f:
    lines = f.readlines()
label_count = lines.count('}\n')

In [19]:
update_pipeline_config(
    config_path=files['PIPELINE_CONFIG'],
    label_map_path=files['LABELMAP'],
    train_record_path=os.path.join(paths['ANNOTATION_PATH'], name + '_train.record'),
    test_record_path=os.path.join(paths['ANNOTATION_PATH'], name + '_test.record'),
    checkpoint_path=paths['CHECKPOINT_PATH']
)

In [14]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline_config)  

ParseError: 172:3 : Message type "object_detection.protos.TrainConfig" has no field named "fine_tune_checkpoint_version".

In [15]:
files['LABELMAP']

'Tensorflow/workspace/annotations/label_map_karp.pbtxt'

In [16]:
with open(files['LABELMAP']) as f:
    lines = f.readlines()
label_count = lines.count('}\n')

In [17]:
label_count

14

In [18]:
pipeline_config.model.ssd.num_classes = label_count
pipeline_config.train_config.batch_size = 4
pipeline_config.train_config.fine_tune_checkpoint = os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'checkpoint', 'ckpt-0')
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path= files['LABELMAP']
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [os.path.join(paths['ANNOTATION_PATH'], name + '_train.record')]
pipeline_config.eval_input_reader[0].label_map_path = files['LABELMAP']
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [os.path.join(paths['ANNOTATION_PATH'], name + '_test.record')]

In [19]:
config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "wb") as f:                                                                                                                                                                                                                     
    f.write(config_text)   

# Обучение модели

In [21]:
TRAINING_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'model_main_tf2.py')

In [22]:
command = "python3 {} --model_dir={} --pipeline_config_path={} --num_train_steps=2000".format(TRAINING_SCRIPT, paths['CHECKPOINT_PATH'],files['PIPELINE_CONFIG'])

Запустить через терминал (внутри venv!)

In [23]:
print(command)

python3 Tensorflow/models/research/object_detection/model_main_tf2.py --model_dir=Tensorflow/workspace/models/my_ssd_mobnet/karp_checkpoints --pipeline_config_path=Tensorflow/workspace/models/my_ssd_mobnet/karp_checkpoints/karp_pipeline.config --num_train_steps=2000


In [29]:
%run Tensorflow/models/research/object_detection/model_main_tf2.py --model_dir=Tensorflow/workspace/models/my_ssd_mobnet --pipeline_config_path=Tensorflow/workspace/models/my_ssd_mobnet/pipeline.config --num_train_steps=2000

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


2024-03-04 11:59:50.825219: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-04 11:59:50.935897: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-04 11:59:50.936425: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

INFO:tensorflow:Maybe overwriting train_steps: 2000


I0304 11:59:51.115087 140160170041344 config_util.py:552] Maybe overwriting train_steps: 2000


INFO:tensorflow:Maybe overwriting use_bfloat16: False


...

W0304 11:59:59.429207 140160170041344 deprecation.py:50] From /home/midnightknight/Blood-object-detection/obj_det/lib/python3.10/site-packages/tensorflow/python/util/dispatch.py:1260: to_float (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.cast` instead.
2024-03-04 12:00:02.318178: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 109495296 exceeds 10% of free system memory.
2024-03-04 12:00:02.437699: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 109495296 exceeds 10% of free system memory.
2024-03-04 12:00:02.441547: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 109495296 exceeds 10% of free system memory.
2024-03-04 12:00:02.459641: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 109495296 exceeds 10% of free system memory.
2024-03-04 12:00:02.501806: W external/local_tsl/tsl/framework/cpu_alloc

W0304 12:00:23.688475 140160170041344 checkpoint.py:225] Detecting that an object or model or tf.train.Checkpoint is being deleted with unrestored values. See the following logs for the specific values in question. To silence these warnings, use `status.expect_partial()`. See https://www.tensorflow.org/api_docs/python/tf/train/Checkpoint#restorefor details about the status object returned by the restore function.


W0304 12:00:23.689536 140160170041344 checkpoint.py:234] Value in checkpoint could not be found in the restored object: (root).optimizer.iter


W0304 12:00:23.690171 140160170041344 checkpoint.py:234] Value in checkpoint could not be found in the restored object: (root).optimizer.decay


W0304 12:00:23.690841 140160170041344 checkpoint.py:234] Value in checkpoint could not be found in the restored object: (root).optimizer.momentum


Нужно опять поменять код внутри библиотеки
https://github.com/tensorflow/models/issues/11099

Внизу команда для терминала, которая позволяет посмотреть показатели обучения

# Оценивание модели

In [15]:
command = "python3 {} --model_dir={} --pipeline_config_path={} --checkpoint_dir={}".format(TRAINING_SCRIPT, paths['CHECKPOINT_PATH'],files['PIPELINE_CONFIG'], paths['CHECKPOINT_PATH'])

Запустить через терминал (внутри venv!)

In [16]:
print(command)

python3 Tensorflow/models/research/object_detection/model_main_tf2.py --model_dir=Tensorflow/workspace/models/my_ssd_mobnet --pipeline_config_path=Tensorflow/workspace/models/my_ssd_mobnet/pipeline.config --checkpoint_dir=Tensorflow/workspace/models/my_ssd_mobnet
